In [2]:
"""
Created on Wed Jul 5 2023

@author: Laia Amorós

Version that Maija has modified
"""

import netCDF4 as nc
import numpy as np
import xarray as xr

import os
import sys
#sys.path.append('./PriorCovCalc/')
#from nearestPD import nearestPD

#sys.path.append('../src')

def regrid(data, lon, lat, factor1, factor2):
    """
    Regrids data to lower resolution by averaging over a window of size factor1 x factor2.
    """
    data_unmasked = data.filled(np.nan)
    data_shape = data.shape
    regridded_shape = (data_shape[0] // factor1, data_shape[1] // factor2)

    data_reg = np.full(regridded_shape, np.nan)
    lon_reg = np.zeros(regridded_shape)
    lat_reg = np.zeros(regridded_shape)

    for i in range(regridded_shape[0]):
        for j in range(regridded_shape[1]):
            window = data_unmasked[i*factor1:min((i+1)*factor1, data_shape[0]), 
                                                j*factor2:min((j+1)*factor2, data_shape[1])]
            lon_window = lon[i*factor1:min((i+1)*factor1, data_shape[0]), 
                                                j*factor2:min((j+1)*factor2, data_shape[1])]
            lat_window = lat[i*factor1:min((i+1)*factor1, data_shape[0]), 
                                                j*factor2:min((j+1)*factor2, data_shape[1])]

            data_reg[i, j] = np.nanmean(window)
            lon_reg[i, j] = np.nanmean(lon_window)
            lat_reg[i, j] = np.nanmean(lat_window)

    # Mask new arrays where np.isnan is True
    data_reg = np.ma.masked_where(np.isnan(data_reg), data_reg)

    return data_reg, lon_reg, lat_reg


def covariance_matrix(matrix):
    """
    Computes the covariance matrix of a matrix with NaN values.
    """
    matrix_flat = np.asarray(matrix).flatten()
    deviations = matrix_flat - np.nanmean(matrix_flat)
    deviations[np.isnan(deviations)] = 0
    outer_product = np.outer(deviations, deviations)
    num_non_nan = np.count_nonzero(~np.isnan(matrix_flat))
    covariance_matrix = outer_product / (num_non_nan - 1)

    return covariance_matrix

# def ensure_pd(covariance):
#     """Check if matrix positive definite. If not, take nearest PD. OBS! The nearestPD function is
#    extremely slow. Need to change to a faster one (didn't finish in an hour with n ~ 33000) """
#     try:
#         np.linalg.cholesky(covariance)
#     except:
#         print('Maxrix not positive definite. Take nearest PD.')
#         covariance = nearestPD(covariance)
#     return covariance

def create_dataset(covariance, lon, lat):
    out_cov = xr.Dataset(
        data_vars={"covariance": (["nparams", "nparams"], covariance)},
        coords={"lon": (["nparams"], np.asarray(lon).flatten()),
                  "lat": (["nparams"], np.asarray(lat).flatten()),},
        attrs={'comment': "Spatial uncertainty covariance matrix for gridcells numbered in the variable nparams with coordinates of the centre of each gridcell"}
    )
    return out_cov

    


def main():
    NUMBER_OF_FILES = 2 # change to the number of covariance matrices you want to produce
    FACTOR1 = 15
    FACTOR2 = 2

    # Path to CO2M simulations data and output
    # in LUMI
 #   DATA_PATH = '/scratch/project_462000289/CO2M_obs/CO2M_simulations/2018/Orbits_Europe/CO2Meast/'
 #   OUTPUT_PATH = '/scratch/project_462000289/covariance_matrices'
    DATA_PATH = '/home/pietaril/Documents/data/CO2M_testdata/CO2M_simulations/'
    OUTPUT_PATH = '/home/pietaril/Documents/data/CO2M_testdata/unc_cov_matrices/'
    
    filenames = [os.path.join(DATA_PATH, f) for f in os.listdir(DATA_PATH) if f.endswith('.nc')]
        
    for file in filenames[:NUMBER_OF_FILES]:
        # Read data
        data_nc = nc.Dataset(file, 'r')
        xco2 = data_nc.groups['data']['observation_data']['xco2'][:]
        xco2_quality_flag = data_nc.groups['data']['observation_data']['xco2_quality_flag'][:]
        uncertainties = data_nc.groups['data']['observation_data']['xco2_precision'][:]
        lon = data_nc.groups['data']['geolocation_data_dem']['longitude'][:]
        lat = data_nc.groups['data']['geolocation_data_dem']['latitude'][:]

        # Regrid data to lower resolution. Factor1 and factor2 can be changed to any integer
        uncertainties_reg, lon_reg, lat_reg = regrid(uncertainties, lon, lat, FACTOR1, FACTOR2)

        #Compute covariance matrix
        covariance = covariance_matrix(uncertainties_reg)
    
        #make sure the matrix is positive definite
        #covariance = ensure_pd(covariance)


        # create xarray dataset with covariance matrix and corresponding spatial coordinates 
        out_cov = create_dataset(covariance, lon_reg, lat_reg)
        


        # Save covariance matrix and coordinates as xarray.dataset to output folder
        date = os.path.basename(file)[21:29]
        output_filename = f'unc_cov_matrix{date}.nc'
        out_cov.to_netcdf(os.path.join(OUTPUT_PATH, output_filename))
        #np.save(os.path.join(OUTPUT_PATH, output_filename), covariance)


# if __name__ == "__main__":
#     main()



Read the matrix just saved to check if everything ok

In [3]:
ds = xr.open_dataset('/home/pietaril/Documents/data/CO2M_testdata/unc_cov_matrices/unc_cov_matrix20250609_notpd.nc')
ds

<xarray.Dataset>
Dimensions:     (nparams: 33495)
Coordinates:
    lon         (nparams) float64 ...
    lat         (nparams) float64 ...
Dimensions without coordinates: nparams
Data variables:
    covariance  (nparams, nparams) float64 ...
Attributes:
    comment:  Spatial uncertainty covariance matrix for gridcells numbered in...

In [4]:
cov = ds["covariance"].values



In [5]:
cov_flat = cov.flatten()

In [6]:
len(cov_flat)

1121915025

In [7]:
len(cov_flat[~np.isnan(cov_flat)])

1121915025

In [5]:
unc

masked_array(
  data=[[--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        ...,
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --]],
  mask=[[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]],
  fill_value=1e+20)

In [8]:
len(cov[abs(cov) > 0.])

34225

In [11]:
cov.shape

(33495, 33495)

For some reason lat & lon not given in 1D so needed to flatten

In [20]:
np.asarray(lon).flatten()

array([  19.08547592,   19.00862694,   18.93191719, ..., -148.00617981,
       -147.87005615, -147.73410034])

In [17]:
lat.flatten()

(609, 55)